<a href="https://colab.research.google.com/github/Jaejuna/MiraeCity/blob/JJJ/EDA_%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EDA 연습 - 퇴근후딴짓

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train = pd.read_csv('../')

In [ ]:
train.head()

# 캐글 공유 코드 - 오디오 EDA

https://www.kaggle.com/code/coldpumpkin/mfcc-feature-extraction-classification-and-eda



In [ ]:
!pip install librosa

In [ ]:
import numpy as np
import pandas as pd

import os
import glob

import librosa
import librosa.display

from tqdm import tqdm_notebook as tqdm

from keras.models import Model
from keras.utils import np_utils

import IPython
import IPython.display as ipd

import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import matplotlib.pyplot as plt

plt.style.use("ggplot")

In [ ]:
# df_train = pd.read_csv('../input/birdsong-recognition/train.csv')
# df_train

# dataset split 필요

In [ ]:
def mfcc_extract(filename):
    try:
        y, sr  = librosa.load(filename, sr = 44100)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, n_fft=int(0.02*sr),hop_length=int(0.01*sr))
        return mfcc
    except:
        return

In [ ]:
def parse_audio_files(parent_dir, sub_dirs, limit):
    labels = []
    features = []
    for label, sub_dir in enumerate(tqdm(sub_dirs)):
        i = 0
        for fn in glob.glob(os.path.join(parent_dir,sub_dir,"*.mp3")):
            if i >= limit:
                break
            features.append(mfcc_extract(fn))
            labels.append(label)
            i+=1
    return features, labels

In [ ]:
%%time

train_cat_dirs = glob.glob(train_dir+'/*')
train_cat = []
for cat_dir in train_cat_dirs:
    tmp = cat_dir.split('/')[-1]
    train_cat.append(tmp)
print('the number of kinds:', len(train_cat))

class_num = len(train_cat)
features, labels = parse_audio_files(train_dir, train_cat, LIMIT)

In [ ]:
print(len(features))
print(features[0].shape)

In [ ]:
fig = plt.figure(figsize=(28,24))
for i,mfcc in enumerate(tqdm(features[:100])):
    if i%40 < 3 :
        sub = plt.subplot(10,3,i%40+3*(i/40)+1)
        librosa.display.specshow(mfcc,vmin=-700,vmax=300)
        if ((i%40+3*(i/40)+1)%3==0) :
            plt.colorbar()
        sub.set_title(train_cat[labels[i]])
plt.show()

In [ ]:
fig = go.Figure(data=[
    go.Bar(y=species.values, x=species.index,marker_color='deeppink')
])

fig.update_layout(title='type 분포')
fig.show()

## 위 처럼 피처 종류 별로 분포를 그래프로 시각화해서 데이터에 대한 분석 가능

In [ ]:
# sampling rate analysis
rec = train.sampling_rate.value_counts()
fig = go.Figure(data=[
    go.Bar(x=rec.index, y=rec.values,marker_color='deeppink')
])

fig.update_layout(title='Top Recordists')
fig.show()

In [ ]:
# channel (mono, stereo; 아마 없을 )
rec = train.channels.value_counts()
fig = go.Figure(data=[
    go.Bar(x=rec.index, y=rec.values,marker_color='deeppink')
])

fig.update_layout(title='Top Recordists')
fig.show()

In [ ]:
# audio signal length
df=train.length.value_counts()
fig = px.pie(df,df.index,df.values,labels={'index':'length of audio'})
fig.update_layout(title='Length of audio signal')
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))

fig.show()

In [ ]:
## 지리적 분석 (해당 코드는 미국을 집약적으로 분석한 것, 이것도 아마 없을 듯)

fig = go.Figure()
fig.add_trace(go.Scattergeo( #그래프의 종류와 데이터 옵션을 지정
        locationmode = 'USA-states', #위치의 항목을 지도의 영역과 일치시키는 데 사용되는 위치 집합을 결정
        lon = df['longitude'],
        lat = df['latitude'],
        text = df['ebird_code'],
        marker = dict(
            size = df['ebird_code'],
            line_color='rgb(40,40,40)',
            line_width=0.5,
            sizemode = 'area'
        )))


fig.update_layout(
        title_text = 'Bird Samples collected From USA',
        showlegend = True,
        geo = dict(
            scope = 'usa',
            landcolor = 'rgb(217, 217, 217)',
        )
    )

fig.show()

--------------------------------------


In [ ]:
# playing audio
# path="../input/birdsong-recognition/train_audio/"
birds=train.ebird_code.unique()[:6]
file=train[train.ebird_code==birds[0]]['filename'][0]

for i in range(0,2):
    file=train[train.ebird_code==birds[i]]['filename'].values[0]
    audio_path=os.path.join(path,birds[i],file)
    print(birds[i])
    IPython.display.display(ipd.Audio(audio_path))

In [ ]:
# waveform visualization
plt.figure(figsize=(17,20 ))


for i in range(0,6):
    file=train[train.ebird_code==birds[i]]['filename'].values[0]
    audio_path=os.path.join(path,birds[i],file)
    plt.subplot(6,2,i+1)
    x , sr = librosa.load(audio_path) #오디오 파일을 읽어 샘플을 리스트로 반환해줌
    librosa.display.waveplot(x, sr=sr,color='r')
    plt.gca().set_title(birds[i])
    plt.gca().get_xaxis().set_visible(False)

In [ ]:
# spectrogram
plt.figure(figsize=(17,20 ))


for i in range(0,6):
    file=train[train.ebird_code==birds[i]]['filename'].values[0]
    audio_path=os.path.join(path,birds[i],file)
    plt.subplot(6,2,i+1)
    x , sr = librosa.load(audio_path)
    x = librosa.stft(x) # 데이터의 스펙트로그램을 리턴
    Xdb = librosa.amplitude_to_db(abs(x)) # 스펙트로그램을 dB 스케일
    librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')
    plt.gca().set_title(birds[i])
    plt.gca().get_xaxis().set_visible(False)
    plt.colorbar()

In [ ]:
# mel-spectrogram
### n_fft = 2048
win_length = 2048
hop_length = 1024
n_mels = 128

plt.figure(figsize=(17,20 ))


for i in range(0,6):
    file=train[train.ebird_code==birds[i]]['filename'].values[0]
    audio_path=os.path.join(path,birds[i],file)
    plt.subplot(6,2,i+1)
    x , sr = librosa.load(audio_path)
    x = librosa.stft(x) # 데이터의 스펙트로그램을 리턴
    mel_spec = librosa.feature.melspectrogram(S=x, sr=sr, n_mels=n_mels, hop_length=hop_length, win_length=win_length)
    Xdb = librosa.amplitude_to_db(abs(mel_spec)) # 스펙트로그램을 dB 스케일
    librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')
    plt.gca().set_title(birds[i])
    plt.gca().get_xaxis().set_visible(False)
    plt.colorbar()

## feature extraction

여기서부터는 푸리에 변환을 통해 time domain 에서 frequency domain으로 데이터를 바꾸어 특징을 추출하는 기술


In [ ]:
import sklearn
# 시각화를 위해서 스펙트럼 중심을 정규화 하는 함수
def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)

In [ ]:
#### Spectral centroid
plt.figure(figsize=(17,20 ))

for i in range(0,6):
    file=train[train.ebird_code==birds[i]]['filename'].values[0]
    audio_path=os.path.join(path,birds[i],file)
    plt.subplot(6,2,i+1)
    x , sr = librosa.load(audio_path)
    spectral_centroids = librosa.feature.spectral_centroid(x, sr=sr)[0]
    frames = range(len(spectral_centroids))
    t = librosa.frames_to_time(frames)
    librosa.display.waveplot(x, sr=sr, alpha=0.4)
    plt.plot(t, normalize(spectral_centroids), color='b')
    plt.gca().set_title(birds[i])
    plt.gca().get_xaxis().set_visible(False)

In [ ]:
### zero crossing rate
x , sr = librosa.load(audio_path)
plt.figure(figsize=(14, 5))
librosa.display.waveplot(x, sr=sr)
# Zooming in
n0 = 9000
n1 = 9100
plt.figure(figsize=(14, 5))
plt.plot(x[n0:n1])
plt.grid()

In [ ]:
zero_crossings = librosa.zero_crossings(x[n0:n1], pad=False)
print(sum(zero_crossings))

In [ ]:
#### MFCC
plt.figure(figsize=(17, 20))

for i in range(0,6):
    file=train[train.ebird_code==birds[i]]['filename'].values[0]
    audio_path=os.path.join(path,birds[i],file)
    plt.subplot(6,2,i+1)
    x , sr = librosa.load(audio_path)
    mfccs = librosa.feature.mfcc(x, sr=sr)
    librosa.display.specshow(mfccs, sr=sr, x_axis='time')
    plt.gca().set_title(birds[i])
    plt.gca().get_xaxis().set_visible(False)

In [ ]:
### Chroma feature
plt.figure(figsize=(17, 20))

for i in range(0,6):
    file=train[train.ebird_code==birds[i]]['filename'].values[0]
    audio_path=os.path.join(path,birds[i],file)
    plt.subplot(6,3,i+1)
    x , sr = librosa.load(audio_path)
    chromagram = librosa.feature.chroma_stft(x, sr=sr)
    librosa.display.specshow(chromagram, x_axis='time', y_axis='chroma')
    plt.gca().set_title(birds[i])
    plt.gca().get_xaxis().set_visible(False)

In [ ]:
fig=plt.figure(figsize=(15,15))
k=1
for i in range(5):

    file=train[train.ebird_code==birds[i]]['filename'].values[0]
    audio_path=os.path.join(path,birds[i],file)
    plt.subplot(5,3,k)
    k+=1
    x , sr = librosa.load(audio_path)
    s = librosa.stft(x)
    Xdb = librosa.amplitude_to_db(abs(s))
        librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')
    plt.gca().set_title('Spectrogram')
    plt.gca().set_ylabel(birds[i])
    plt.gca().get_xaxis().set_visible(False)

    plt.subplot(5,3,k)
    k+=1
    mfccs = librosa.feature.mfcc(x, sr=sr)
    librosa.display.specshow(mfccs, sr=sr, x_axis='time')
    plt.gca().set_title('MFFC features ')
    plt.gca().get_xaxis().set_visible(False)
        plt.subplot(5,3,k)
    k+=1
    chromagram = librosa.feature.chroma_stft(x, sr=sr)
    librosa.display.specshow(chromagram, x_axis='time', y_axis='chroma', cmap='coolwarm')
    plt.gca().set_title('Chroma feature')
    plt.gca().get_xaxis().set_visible(False)

#fig.suptitle('Comparing audio features for bird species')
plt.tight_layout()
plt.show()

# 캐글 공유 코드 - 이미지 EDA
https://www.kaggle.com/code/dariussingh/nfl-multipose-estimation-using-yolov8-mediapipe 는 참고만
<br/>
especially pose estimation EDA

chatGPT로 EDA 구현

**possible features**:
Raw keypoints, Relative distances, agngles, temporal changes, Velocity, acceleration, joint relationships, symmetry, historical data, pose embeddings

In [ ]:
import cv2
import matplotlib.pyplot as plt

# Load an image
img = cv2.imread('path_to_your_image.jpg')

# Convert from BGR to RGB
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Display the image
plt.imshow(img)
plt.show()

In [ ]:
# Assuming 'labels' is a list of your labels
import pandas as pd

# Convert to a pandas Series for easier frequency calculation
labels_series = pd.Series(labels)

# Show the distribution of labels
print(labels_series.value_counts())

In [ ]:
# Assuming 'keypoints' is a list of your keypoints
keypoints_series = pd.Series(keypoints)

# Show the distribution of keypoints
print(keypoints_series.describe())

In [ ]:
from torchvision import transforms

# Define a transform
transform = transforms.RandomHorizontalFlip()

# Apply the transform to the image
transformed_img = transform(img)

# Display the original and transformed images
fig, ax = plt.subplots(1, 2)
ax[0].imshow(img)
ax[1].imshow(transformed_img)
plt.show()

--------------------------------------------------------------
will perform feature extraction and EDA

# 캐글 공유 코드 - 멀티모달 EDA